In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings

c:\Users\diego\AppData\Local\Programs\Python\Python38\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Cargar el modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Leer el archivo CSV
file_path = "./Data/Entrevista_LuisaGonzales_JuanCueva.csv"
data = pd.read_csv(file_path, sep=",")  # Asegúrate de usar el separador adecuado (tabulación en este caso)

In [3]:
data

,ID,Candidato,Temas,Descripción,Entrevista
0,LG1,Luisa Gonzales,justicia social propuesta corrupción,entrevista relevante entender propuesta visión...,buen día lenin saludo escuchar momento revoluc...
1,LG2,Luisa Gonzales,crisis eléctrico relación internacional acción,entrevista ofrecer visión claro prioridad estr...,mucho gracia buen día ecuatoriano mirar moment...
2,LG3,Luisa Gonzales,venezuela rafael correo persecución,gonzález defender independencia capacidad lide...,querido fernando necesitar consultar ninguno a...
3,JC1,Juan Cueva,propiedad intelectual panorama ecuatoriano,abordo tema clave relacionado protección gesti...,propiedad intelectual derecho cualquiera autor...
4,JC2,Juan Cueva,proyecto político experiencia corrupción,entrevista juan iván cueva candidato presidenc...,mucho gracia lenin invitación honor aquí prese...
5,JC3,Juan Cueva,colaboración sector desafío protección caso éx...,entrevista juan iván cueva compartir experienc...,estrategia considerar efectivo ver protección ...


In [4]:
# Extraer las columnas de interés
documents = data['Entrevista'].tolist()  # El texto del documento
ids = data['ID'].tolist()  # Identificadores únicos
metadatas = data[['Candidato', 'Temas', 'Descripción']].to_dict(orient='records')  # Metadatos

In [5]:
# Generar los embeddings para los documentos
embeddings = model.encode(documents).tolist()

In [6]:
# Inicializar la base de datos vectorial con persistencia
client = chromadb.PersistentClient(
    path="./chroma_db"  # Carpeta donde se guardarán los datos
)

In [7]:
# Crear o cargar la colección
collection = client.get_or_create_collection("prueba_collection")

# Agregar los documentos, metadatos, IDs y embeddings a la colección
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids,
    embeddings=embeddings
)

print("Documentos agregados exitosamente a la colección.")

Insert of existing embedding ID: LG1
Insert of existing embedding ID: LG2
Insert of existing embedding ID: LG3
Insert of existing embedding ID: JC1
Insert of existing embedding ID: JC2
Insert of existing embedding ID: JC3
Add of existing embedding ID: LG1
Add of existing embedding ID: LG2
Add of existing embedding ID: LG3
Add of existing embedding ID: JC1
Add of existing embedding ID: JC2
Add of existing embedding ID: JC3


Documentos agregados exitosamente a la colección.


In [8]:
# Realizar una consulta
query = "corrupción"  # Tu consulta en texto
query_embedding = model.encode([query]).tolist()  # Generar el embedding de la consulta

# Consultar los documentos más similares
results = collection.query(
    query_embeddings=query_embedding,
    n_results=3  # Devuelve los 3 documentos más similares
)

# Imprimir los resultados
print("Resultados de la consulta:")
for result in results['documents']:
    print(result)

Resultados de la consulta:
['querido fernando necesitar consultar ninguno asesor manejar comunicación mujer capaz inteligente ir ver desafío mostrar cuántos país mundo violar derechos humanos mantener relación comercial unidos ojalá poder dialogar país violar derechos humanos comerciar unidos solo mencionar venezuelo habl palabra aislado cumbre iberoamericano nación 2023 ecuador anfitrión ninguno gobierno asistir aislado igual relación roto méxico socio comercial clave ecuador cerrar 2024 crecimiento 22 pobreza extremo pobreza afectar casi mitad población jugar vida elección debate permitir exponer plan trabajo atacar rafael correo problema país llamar unidad dejar confrontación odio si daniel novoa aceptar invitar debate confrontar mentira verdad demostrar plan estructurado ecuador 16 candidato 15 mencionar correa propuesta claro rafael correa líder histórico revolución ciudadanar hombre inocente presidenta partido dirijo campaña gobierno ecuador gobernar justicia ecuatoriano apenas 1

In [9]:
# Listar las colecciones existentes
collections = client.list_collections()

# Imprimir las colecciones
for collection in collections:
    print("Nombre de la colección:", collection.name)

Nombre de la colección: prueba_collection


In [10]:
# Obtener la colección por nombre
collection = client.get_collection("prueba_collection")

# Recuperar todos los datos (embeddings, IDs y metadatos)
datos = collection.get()

# Imprimir el contenido
print("IDs:", datos["ids"])
print("Embeddings:", datos["embeddings"])
print("Metadatos:", datos["metadatas"])


IDs: ['LG1', 'LG2', 'LG3', 'JC1', 'JC2', 'JC3']
Embeddings: None
Metadatos: [{'Candidato': 'Luisa Gonzales', 'Descripción': 'entrevista relevante entender propuesta visión político luisa gonzález contexto crisis económico social seguridad ecuador', 'Temas': 'justicia social propuesta corrupción'}, {'Candidato': 'Luisa Gonzales', 'Descripción': 'entrevista ofrecer visión claro prioridad estrategia luisa gonzález enfrentar desafío económico social político ecuador', 'Temas': 'crisis eléctrico relación internacional acción'}, {'Candidato': 'Luisa Gonzales', 'Descripción': 'gonzález defender independencia capacidad liderar criticar justicia ecuatoriano enfatizar necesidad unidad liderazgo superar crisis país además responder pregunta relación rafael correa corrupción visión política exterior', 'Temas': 'venezuela rafael correo persecución'}, {'Candidato': 'Juan Cueva', 'Descripción': 'abordo tema clave relacionado protección gestión derecho propiedad intelectual explicar concepto fundament